### Setup

In [89]:
##Matplot lib and use the `nbagg` backend DELETE THIS COMMENT.
import matplotlib
matplotlib.use('nbagg') #DELETE?
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt
import pandas as pd

In [90]:
# Import SQLAlchemy `automap` and other dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [91]:
# Create an engine for the `emoji.sqlite` database
#http://docs.sqlalchemy.org/en/latest/core/engines.html - echo is false so that we don't have logging.
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

## Reflect database and Query

In [92]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Stations = Base.classes.stations

In [93]:
Measurements = Base.classes.measurements

In [94]:
# Start a session to query the database
session = Session(engine)

### Precipitation Analysis

In [96]:
from sqlalchemy import Date, cast
from datetime import datetime,timedelta, date #as dt
from dateutil.relativedelta import relativedelta
import matplotlib.dates as dts
#from datetime import datetime
import datetime as dt
from dateutil.relativedelta import relativedelta

In [102]:
maximumdate = session.query(func.max(Measurements.date))
#lastyearmaxdt = maximumdate - relativedelta(months=12)

In [104]:
last_date = session.query(Measurements.date).order_by(Measurements.date.desc()).first()
print(last_date)

None


In [107]:
query_begin_date = session.query(Measurements.date).order_by(Measurements.date.desc()).filter (Measurements.station =='USC00511918').first()
print(query_begin_date)

None


In [108]:
conn.execute("select * from Measurements limit 5").fetchall()

NameError: name 'conn' is not defined

In [79]:
# Latest Date, this is my vacation end date
#latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
latest_date = session.query(Measurements.date).\
filter(Measurements.station == 'USC00511918').\
order_by(Measurements.date.desc()).first()


# filter(Measurement.station == 'USC00511918').\
# order_by(Measurement.date.desc()).first()

In [80]:
print(latest_date)
#session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= '2017-01-01')

None


In [32]:
datetime.strftime

<method 'strftime' of 'datetime.date' objects>

In [76]:
#Date 12 months ago
y = int(latest_date[0][:4])
m = int(latest_date[0][5:7])
d = int(latest_date[0][8:10])
print(y)
print(m)
print(d)
start_date = dt.date(y, m, d) - dt.timedelta(days = 365)
print(start_date)

TypeError: 'NoneType' object is not subscriptable

In [75]:
# start_date = session.query(Measurements.date).\
# filter(Measurements.station == 'USC00511918').\
# order_by(Measurements.date.asc()).first()

# start_date = latest_date - str(365)
# .strftime('%Y-%m-%d')
# start_date = session.query(date(Measurements.date, '+365 day'))

#start_date = dlatest_date(y, m, d) - dt.timedelta(days = 365)
start_date = dt.date(y, m, d) - dt.timedelta(days = 365)

NameError: name 'y' is not defined

In [70]:
from datetime import datetime, timedelta
futuredate = datetime.now() + timedelta(days=10)

In [72]:
   #current_time = datetime.now()
current_time = datetime.now()
past_year = current_time - datetime.timedelta(days=365)
#past_year = latest_date - datetime.timedelta(days=365)

#     measurements_year = session.query(Measurements.date,Measurements.prcp).filter(
#         Measurements.date > past_year).all()

AttributeError: type object 'datetime.datetime' has no attribute 'timedelta'

In [60]:
datetime_obj = datetime.strptime(latest_date, "%Y-%m-%d")
datetime_obj                            

TypeError: strptime() argument 1 must be str, not None

In [85]:
# Designed a query to retrieve the last 12 months of precipitation data.
# precip = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= today()-365)
# '2017-01-01')
# precip = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= '2017-01-01')
#Max date is 8/23/2017.
precip = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= '2016-08-23')

#LAST TRY:
# precip = session.query(Measurements.date, Measurements.prcp).\
#    filter(Measurements.date>= start_date).all()

# precip = session.query(Measurements.date, Measurements.prcp).filter(cast(date.date_time,Date)>= today()-365)
#https://stackoverflow.com/questions/7075828/make-sqlalchemy-use-date-in-filter-using-postgresql

# my_data = session.query(MyObject).\
# filter(cast(MyObject.date_time,Date) == date.today()).all()

In [87]:
precip

In [26]:
# from sqlalchemy import Date, cast
# from datetime import datetime

lastyrdt = datetime.now().date() 
lastyrdt 
# https://stackoverflow.com/questions/16228195/comparing-dates-with-sqlalchemy-sqlite
#query.filter(cast(Model.created_at, Date) == cast(created_at,Date))

datetime.date(2018, 4, 22)

In [36]:
# from datetime import strftime 
import time

In [38]:
testdt = time.strftime('%Y-%m-%d',Measurements.date) >= date('now','-365 days')
testdt
# TypeError: Tuple or struct_time argument required

TypeError: Tuple or struct_time argument required

In [42]:
latest_date = session.query(Measurements.date).\
order_by(Measurements.date.desc()).first()

In [45]:
# print(latest_date)

# ldt = pd.read_sql_query(latest_date, session.bind)
# ldt

In [41]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [86]:
# create empty dict to populate
data = {'date': [], 'precip': []}

# populate dict with rows from results
for row in precip:
    data['date'].append(row.date)
    data['prcp'].append(row.precip)

# create pandas df from dict
precip_df = pd.DataFrame(data['precip'], columns = ['prcp'], index = data['date'])
precip_df.head()

,prcp
